In [1]:
demanda = [5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 12,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 5,
 5,
 5,
 5,
 5,
 6,
 5,
 5,
 5,
 5,
 5,
 7,
 11,
 5,
 5,
 7,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 10,
 19,
 11,
 5,
 10,
 5,
 6,
 5,
 7,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 5,
 5,
 6,
 5,
 5,
 5,
 5,
 5,
 8,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 7,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 8,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 7,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 11,
 5,
 6,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5]

### Codigo 2

In [28]:
import random
import numpy as np

# Datos de ejemplo: estaciones y demanda
num_estaciones = 300
demanda_estaciones = demanda[:]
bicicletas_disponibles = 1500

# Parámetros de ACO
num_hormigas = 50
num_iteraciones = 100
alfa = 1.0  # Influencia de feromonas
beta = 1.0  # Influencia heurística
ro = 0.1  # Tasa de evaporación de feromonas
Q = 100  # Cantidad de feromonas depositadas por cada hormiga
feromonas = np.ones((num_estaciones, num_estaciones))

# Función para calcular la probabilidad de transferencia de bicicletas
def calcular_probabilidad_transferencia(hormiga, estacion_actual, disponibles):
    probabilidad = []
    for estacion in disponibles:
        if demanda_estaciones[estacion] > 0:
            prob = (feromonas[estacion_actual][estacion] ** alfa) * ((1.0 / demanda_estaciones[estacion]) ** beta)
            probabilidad.append(prob)
    total_probabilidad = sum(probabilidad)
    return [p / total_probabilidad for p in probabilidad]

# Función para evaluar la calidad de la asignación
def evaluar_asignacion(asignacion, demanda_estaciones):
    costo_total = 0
    exceso_total = 0

    for estacion, bicicletas_asignadas in enumerate(asignacion):
        demanda = demanda_estaciones[estacion]
        exceso = bicicletas_asignadas - demanda  # Calcula el exceso de bicicletas en la estación

        # Penalización por exceso de bicicletas
        if exceso >= 1:
            costo_total += exceso

        exceso_total += exceso

    return costo_total, exceso_total

# Inicializa el costo, el exceso y el número de bicicletas asignadas
mejor_costo = float('inf')
mejor_exceso = float('inf')
mejor_asignacion = np.zeros(num_estaciones)
bicicletas_asignadas_total = 0

# Algoritmo de optimización de colonia de hormigas
for _ in range(num_iteraciones):
    for hormiga in range(num_hormigas):
        estacion_actual = random.randint(0, num_estaciones - 1)
        disponibles = list(range(num_estaciones))
        disponibles.remove(estacion_actual)
        asignacion_hormiga = [0] * num_estaciones

        while disponibles:
            probabilidad = calcular_probabilidad_transferencia(hormiga, estacion_actual, disponibles)

            # Verifica si hay probabilidades válidas (mayores a cero)
            probabilidades_validas = [p for p in probabilidad if p > 0]

            if len(probabilidades_validas) == 0:
                # Si no hay probabilidades válidas, selecciona una estación aleatoria de las disponibles
                siguiente_estacion = np.random.choice(disponibles)
            else:
                # Si hay probabilidades válidas, selecciona con base en las probabilidades
                probabilidad_normalizada = [p / sum(probabilidades_validas) for p in probabilidades_validas]
                siguiente_estacion = np.random.choice([e for e, p in zip(disponibles, probabilidad) if p > 0], p=probabilidad_normalizada)

            bicicletas_transferidas = min(bicicletas_disponibles - bicicletas_asignadas_total, demanda_estaciones[siguiente_estacion])
            asignacion_hormiga[siguiente_estacion] = bicicletas_transferidas
            bicicletas_asignadas_total += bicicletas_transferidas

            # Actualizar feromonas basadas en la transferencia
            for i in range(num_estaciones):
                estacion1 = estacion_actual
                estacion2 = siguiente_estacion
                feromonas[estacion1][estacion2] = (1 - ro) * feromonas[estacion1][estacion2] + Q

            estacion_actual = siguiente_estacion
            disponibles.remove(siguiente_estacion)

    # Evaluar la calidad de la asignación
    asignacion_actual = np.argmax(feromonas, axis=1)
    costo, exceso = evaluar_asignacion(asignacion_actual, demanda_estaciones)
    
    # Comparar con la mejor asignación y costo anterior
    if costo < mejor_costo and exceso >= 0:
        mejor_asignacion = asignacion_actual
        mejor_costo = costo
        mejor_exceso = exceso

   # Resultado final
print("¡Mejor asignación encontrada!")
print("Mejor asignación de bicicletas:", mejor_asignacion.tolist())
print("Costo final:", mejor_costo)
print("Exceso total:", mejor_exceso)


¡Mejor asignación encontrada!
Mejor asignación de bicicletas: [3, 3, 0, 2, 6, 1, 16, 8, 10, 10, 0, 2, 18, 4, 0, 4, 2, 0, 13, 21, 4, 3, 7, 2, 19, 24, 0, 2, 7, 0, 0, 5, 7, 1, 1, 6, 16, 8, 11, 6, 2, 8, 8, 2, 8, 0, 8, 0, 0, 9, 8, 5, 5, 6, 2, 6, 13, 1, 4, 6, 11, 11, 3, 45, 3, 4, 5, 3, 7, 2, 5, 2, 3, 0, 2, 9, 0, 9, 9, 1, 0, 18, 0, 5, 0, 0, 0, 18, 8, 1, 13, 3, 10, 1, 0, 1, 0, 31, 2, 0, 1, 11, 21, 5, 4, 5, 11, 0, 2, 0, 1, 2, 9, 8, 3, 10, 1, 6, 0, 11, 0, 10, 3, 2, 8, 0, 11, 2, 4, 2, 8, 2, 5, 2, 1, 2, 6, 1, 3, 0, 16, 6, 12, 3, 0, 0, 9, 8, 9, 1, 1, 3, 5, 0, 8, 0, 3, 1, 0, 4, 5, 2, 9, 8, 10, 3, 5, 4, 2, 3, 9, 3, 4, 1, 2, 1, 8, 3, 13, 6, 7, 0, 9, 1, 0, 3, 0, 13, 10, 6, 3, 2, 3, 2, 1, 3, 12, 0, 5, 7, 16, 24, 2, 4, 6, 2, 7, 1, 1, 6, 3, 8, 10, 4, 0, 5, 5, 3, 4, 1, 9, 4, 15, 0, 14, 14, 14, 10, 1, 1, 8, 12, 1, 3, 4, 5, 8, 6, 5, 7, 10, 6, 1, 1, 0, 8, 1, 1, 2, 1, 14, 3, 0, 5, 9, 8, 3, 8, 16, 4, 4, 3, 12, 1, 2, 8, 2, 9, 1, 13, 1, 0, 10, 4, 7, 10, 1, 8, 0, 2, 0, 2, 3, 0, 0, 0, 11, 3, 0, 9, 3, 2, 27, 0, 4, 1

In [4]:
import random
import numpy as np

# Datos de ejemplo: estaciones y demanda
num_estaciones = 300
demanda_estaciones = demanda[:]
bicicletas_disponibles = 1500

# Parámetros de ACO
num_hormigas = 50
num_iteraciones = 100
alfa = 1.0  # Influencia de feromonas
beta = 1.0  # Influencia heurística
ro = 0.1  # Tasa de evaporación de feromonas
Q = 100  # Cantidad de feromonas depositadas por cada hormiga
feromonas = np.ones((num_estaciones, num_estaciones))

# Función para calcular la probabilidad de transferencia de bicicletas
def calcular_probabilidad_transferencia(hormiga, estacion_actual, disponibles, bicicletas_disponibles):
    probabilidad = []
    for estacion in disponibles:
        if demanda_estaciones[estacion] > 0:
            prob = (feromonas[estacion_actual][estacion] ** alfa) * ((1.0 / demanda_estaciones[estacion]) ** beta)
            probabilidad.append(prob)
    total_probabilidad = sum(probabilidad)
    return [p / total_probabilidad for p in probabilidad]

# Función para evaluar la calidad de la asignación
def evaluar_asignacion(asignacion, demanda_estaciones):
    costo_total = 0
    exceso_total = 0

    for estacion, bicicletas_asignadas in enumerate(asignacion):
        demanda = demanda_estaciones[estacion]
        exceso = bicicletas_asignadas - demanda  # Calcula el exceso de bicicletas en la estación

        # Penalización por exceso de bicicletas
        if exceso >= 1:
            costo_total += exceso

        exceso_total += exceso

    return costo_total, exceso_total

# Inicializa el costo, el exceso y el número de bicicletas asignadas
mejor_costo = float('inf')
mejor_exceso = float('inf')
mejor_asignacion = np.zeros(num_estaciones)
bicicletas_asignadas_total = 0

# Algoritmo de optimización de colonia de hormigas
for _ in range(num_iteraciones):
    for hormiga in range(num_hormigas):
        estacion_actual = random.randint(0, num_estaciones - 1)
        disponibles = list(range(num_estaciones))
        disponibles.remove(estacion_actual)
        asignacion_hormiga = [0] * num_estaciones

        while disponibles:
            probabilidad = calcular_probabilidad_transferencia(hormiga, estacion_actual, disponibles, bicicletas_disponibles - bicicletas_asignadas_total)

            # Verifica si hay probabilidades válidas (mayores a cero)
            probabilidades_validas = [p for p in probabilidad if p > 0]

            if len(probabilidades_validas) == 0:
                # Si no hay probabilidades válidas, selecciona una estación aleatoria de las disponibles
                siguiente_estacion = np.random.choice(disponibles)
            else:
                # Si hay probabilidades válidas, selecciona con base en las probabilidades
                probabilidad_normalizada = [p / sum(probabilidades_validas) for p in probabilidades_validas]
                siguiente_estacion = np.random.choice([e for e, p in zip(disponibles, probabilidad) if p > 0], p=probabilidad_normalizada)

            bicicletas_transferidas = min(bicicletas_disponibles - bicicletas_asignadas_total, demanda_estaciones[siguiente_estacion])
            asignacion_hormiga[siguiente_estacion] = bicicletas_transferidas
            bicicletas_asignadas_total += bicicletas_transferidas

            # Actualizar feromonas basadas en la transferencia
            for i in range(num_estaciones):
                estacion1 = estacion_actual
                estacion2 = siguiente_estacion
                feromonas[estacion1][estacion2] = (1 - ro) * feromonas[estacion1][estacion2] + Q

            estacion_actual = siguiente_estacion
            disponibles.remove(siguiente_estacion)

        # Evaluar la calidad de la asignación de la hormiga
        asignacion_actual = np.argmax(feromonas, axis=1)
        costo, exceso = evaluar_asignacion(asignacion_actual, demanda_estaciones)
        
        # Comparar con la mejor asignación y costo anterior
        if costo < mejor_costo and exceso >= 0:
            mejor_asignacion = asignacion_actual
            mejor_costo = costo
            mejor_exceso = exceso

        # Comprobar si se han asignado todas las bicicletas disponibles
        if bicicletas_asignadas_total >= bicicletas_disponibles:
            break

    # Comprobar si se han asignado todas las bicicletas disponibles
    if bicicletas_asignadas_total >= bicicletas_disponibles:
        break

# Resultado final
print("¡Mejor asignación encontrada!")
print("Mejor asignación de bicicletas:", mejor_asignacion.tolist())
print("Costo final:", mejor_costo)
print("Exceso total:", mejor_exceso)


¡Mejor asignación encontrada!
Mejor asignación de bicicletas: [77, 95, 158, 70, 105, 79, 65, 46, 186, 236, 292, 152, 167, 110, 178, 119, 273, 35, 245, 211, 134, 149, 169, 26, 60, 170, 284, 146, 271, 144, 296, 126, 15, 109, 127, 68, 97, 263, 243, 234, 0, 282, 121, 228, 32, 40, 123, 253, 0, 256, 44, 235, 195, 168, 189, 101, 184, 76, 247, 56, 98, 92, 72, 261, 41, 59, 155, 80, 91, 140, 239, 166, 264, 268, 241, 276, 221, 23, 106, 17, 10, 58, 274, 248, 114, 50, 30, 37, 200, 6, 269, 161, 85, 22, 13, 4, 287, 34, 218, 213, 187, 224, 111, 124, 222, 278, 260, 297, 229, 188, 150, 208, 136, 116, 180, 62, 54, 28, 27, 286, 283, 190, 165, 82, 207, 67, 8, 175, 217, 142, 185, 262, 5, 216, 251, 272, 174, 154, 172, 2, 71, 201, 115, 99, 204, 3, 96, 198, 125, 163, 203, 63, 244, 129, 259, 171, 16, 299, 242, 75, 120, 102, 223, 11, 267, 252, 164, 197, 179, 57, 182, 214, 143, 226, 29, 133, 205, 153, 132, 258, 130, 49, 250, 191, 232, 118, 48, 7, 74, 240, 160, 275, 289, 112, 277, 225, 107, 78, 24, 181, 288, 100, 

In [9]:
import random
import numpy as np

# Datos de ejemplo: estaciones y demanda
num_estaciones = 300
demanda_estaciones = demanda[:]
bicicletas_disponibles = 1500

# Parámetros de ACO
num_hormigas = 50
num_iteraciones = 100
alfa = 1.0  # Influencia de feromonas
beta = 1.0  # Influencia heurística
ro = 0.1  # Tasa de evaporación de feromonas
Q = 100  # Cantidad de feromonas depositadas por cada hormiga
feromonas = np.ones((num_estaciones, num_estaciones))

# Función para calcular la probabilidad de transferencia de bicicletas
def calcular_probabilidad_transferencia(hormiga, estacion_actual, disponibles, demanda_estaciones, feromonas):
    probabilidad = []
    for estacion in disponibles:
        if demanda_estaciones[estacion] > 0:
            prob = (feromonas[estacion_actual][estacion] ** alfa) * ((1.0 / demanda_estaciones[estacion]) ** beta)
            probabilidad.append(prob)
    total_probabilidad = sum(probabilidad)
    return [p / total_probabilidad for p in probabilidad]

# Función para evaluar la calidad de la asignación
def evaluar_asignacion(asignacion, demanda_estaciones):
    costo_total = 0
    exceso_total = 0

    for estacion, bicicletas_asignadas in enumerate(asignacion):
        demanda = demanda_estaciones[estacion]
        exceso = bicicletas_asignadas - demanda  # Calcula el exceso de bicicletas en la estación

        # Penalización por exceso de bicicletas
        if exceso >= 1:
            costo_total += exceso

        exceso_total += exceso

    return costo_total, exceso_total

# Inicializa el costo, el exceso y el número de bicicletas asignadas
mejor_costo = float('inf')
mejor_exceso = float('inf')
mejor_asignacion = np.zeros(num_estaciones)

# Algoritmo de optimización de colonia de hormigas
for _ in range(num_iteraciones):
    for hormiga in range(num_hormigas):
        estacion_actual = random.randint(0, num_estaciones - 1)
        disponibles = list(range(num_estaciones))
        disponibles.remove(estacion_actual)
        asignacion_hormiga = [0] * num_estaciones

        while disponibles:
            probabilidad = calcular_probabilidad_transferencia(hormiga, estacion_actual, disponibles, demanda_estaciones, feromonas)

            # Verifica si hay probabilidades válidas (mayores a cero)
            probabilidades_validas = [p for p in probabilidad if p > 0]

            if len(probabilidades_validas) == 0:
                # Si no hay probabilidades válidas, selecciona una estación aleatoria de las disponibles
                siguiente_estacion = np.random.choice(disponibles)
            else:
                # Si hay probabilidades válidas, selecciona con base en las probabilidades
                probabilidad_normalizada = [p / sum(probabilidades_validas) for p in probabilidades_validas]
                siguiente_estacion = np.random.choice([e for e, p in zip(disponibles, probabilidad) if p > 0], p=probabilidad_normalizada)

            # Asegúrate de no asignar más bicicletas de las disponibles en la estación actual
            bicicletas_disponibles_estacion = min(bicicletas_disponibles - sum(asignacion_hormiga), demanda_estaciones[estacion_actual])
            bicicletas_transferidas = min(bicicletas_disponibles_estacion, demanda_estaciones[siguiente_estacion])
            asignacion_hormiga[siguiente_estacion] = bicicletas_transferidas

            # Actualizar feromonas basadas en la transferencia
            for i in range(num_estaciones):
                estacion1 = estacion_actual
                estacion2 = siguiente_estacion
                feromonas[estacion1][estacion2] = (1 - ro) * feromonas[estacion1][estacion2] + Q

            estacion_actual = siguiente_estacion
            disponibles.remove(siguiente_estacion)

        # Evaluar la calidad de la asignación de la hormiga
        asignacion_actual = np.argmax(feromonas, axis=1)
        costo, exceso = evaluar_asignacion(asignacion_actual, demanda_estaciones)

        # Comparar con la mejor asignación y costo anterior
        if costo < mejor_costo and exceso >= 0:
            mejor_asignacion = asignacion_actual
            mejor_costo = costo
            mejor_exceso = exceso

        # Si el costo y el exceso son cercanos a cero, detén el algoritmo
        if costo <= 0.001 and exceso >= 0 and exceso <= 0.001:
            break

# Resultado final
print("¡Mejor asignación encontrada!")
print("Mejor asignación de bicicletas:", mejor_asignacion.tolist())
print("Costo final:", mejor_costo)
print("Exceso total:", mejor_exceso)


¡Mejor asignación encontrada!
Mejor asignación de bicicletas: [6, 7, 3, 2, 7, 11, 1, 5, 0, 4, 1, 0, 13, 1, 1, 3, 10, 1, 1, 7, 11, 5, 5, 0, 8, 2, 4, 0, 5, 24, 1, 0, 2, 15, 2, 14, 3, 9, 2, 14, 0, 0, 6, 1, 0, 7, 0, 4, 0, 2, 7, 9, 2, 2, 1, 3, 0, 0, 9, 1, 0, 8, 2, 3, 6, 1, 8, 5, 1, 2, 8, 1, 28, 25, 15, 3, 3, 16, 0, 4, 6, 0, 2, 0, 9, 0, 5, 3, 4, 2, 10, 6, 3, 11, 7, 0, 2, 8, 3, 8, 2, 5, 12, 12, 0, 0, 4, 0, 7, 1, 3, 24, 2, 11, 17, 1, 6, 2, 1, 2, 10, 0, 6, 3, 2, 2, 1, 2, 0, 2, 3, 8, 8, 9, 6, 0, 3, 4, 0, 5, 0, 2, 7, 2, 3, 5, 9, 23, 0, 1, 3, 8, 5, 3, 11, 1, 0, 1, 9, 13, 6, 1, 15, 1, 5, 0, 10, 1, 1, 7, 2, 7, 8, 23, 4, 0, 5, 15, 0, 4, 4, 3, 1, 4, 12, 9, 2, 5, 6, 13, 4, 2, 22, 0, 4, 3, 27, 1, 10, 0, 7, 1, 2, 1, 6, 1, 5, 5, 8, 15, 5, 2, 0, 8, 2, 3, 1, 4, 10, 3, 15, 6, 15, 1, 0, 6, 0, 4, 5, 5, 5, 6, 1, 14, 1, 1, 4, 2, 3, 8, 1, 3, 13, 10, 19, 5, 3, 2, 10, 7, 4, 21, 7, 8, 6, 9, 2, 1, 0, 1, 0, 4, 0, 3, 9, 6, 17, 0, 2, 5, 15, 0, 0, 17, 1, 1, 1, 3, 7, 8, 22, 1, 11, 3, 4, 0, 9, 3, 1, 1, 27, 8, 4, 3, 8, 6, 2

In [10]:
mj = mejor_asignacion.tolist()
sum(mj)

1576

In [4]:
data2 = pd.read_csv('data/nomenclatura.csv', encoding='ISO-8859-1')

In [5]:
data2

,id,name,obcn,location,latitude,longitude,status
0,2,(GDL-001) C. Epigmenio Glez./ Av. 16 de Sept.,GDL-001,POLÍGONO CENTRAL,20.666378,-103.348820,IN_SERVICE
1,3,(GDL-002) C. Colonias / Av. Niños héroes,GDL-002,POLÍGONO CENTRAL,20.667228,-103.366000,IN_SERVICE
2,4,(GDL-003) C. Vidrio / Av. Chapultepec,GDL-003,POLÍGONO CENTRAL,20.667690,-103.368252,IN_SERVICE
3,5,(GDL-004) C. Ghilardi /C. Miraflores,GDL-004,POLÍGONO CENTRAL,20.691750,-103.362550,IN_SERVICE
4,6,(GDL-005) C. San Diego /Calzada Independencia,GDL-005,POLÍGONO CENTRAL,20.681158,-103.339363,IN_SERVICE
...,...,...,...,...,...,...,...
307,323,(GDL-213) C. Palavicini / Av. Circunvalación,GDL-213,POLÍGONO CENTRAL,20.704670,-103.345589,IN_SERVICE
308,324,(GDL-221) De los científicos/ J. J. Tablada,GDL-221,POLÍGONO CENTRAL,20.700192,-103.345961,IN_SERVICE
309,325,(GDL-226) C. Luis G. Cuevas /Av. Revolución,GDL-226,TLQ-CORREDORATLAS,20.652408,-103.312902,IN_SERVICE
310,326,(GDL-219) C. D. Rivera / C. Filósofos,GDL-219,POLÍGONO CENTRAL,20.698197,-103.342554,IN_SERVICE


In [6]:
data2 = data2.loc[data2['status'] == 'IN_SERVICE']
data2.reset_index(inplace = True)

In [7]:
data2

,index,id,name,obcn,location,latitude,longitude,status
0,0,2,(GDL-001) C. Epigmenio Glez./ Av. 16 de Sept.,GDL-001,POLÍGONO CENTRAL,20.666378,-103.348820,IN_SERVICE
1,1,3,(GDL-002) C. Colonias / Av. Niños héroes,GDL-002,POLÍGONO CENTRAL,20.667228,-103.366000,IN_SERVICE
2,2,4,(GDL-003) C. Vidrio / Av. Chapultepec,GDL-003,POLÍGONO CENTRAL,20.667690,-103.368252,IN_SERVICE
3,3,5,(GDL-004) C. Ghilardi /C. Miraflores,GDL-004,POLÍGONO CENTRAL,20.691750,-103.362550,IN_SERVICE
4,4,6,(GDL-005) C. San Diego /Calzada Independencia,GDL-005,POLÍGONO CENTRAL,20.681158,-103.339363,IN_SERVICE
...,...,...,...,...,...,...,...,...
295,307,323,(GDL-213) C. Palavicini / Av. Circunvalación,GDL-213,POLÍGONO CENTRAL,20.704670,-103.345589,IN_SERVICE
296,308,324,(GDL-221) De los científicos/ J. J. Tablada,GDL-221,POLÍGONO CENTRAL,20.700192,-103.345961,IN_SERVICE
297,309,325,(GDL-226) C. Luis G. Cuevas /Av. Revolución,GDL-226,TLQ-CORREDORATLAS,20.652408,-103.312902,IN_SERVICE
298,310,326,(GDL-219) C. D. Rivera / C. Filósofos,GDL-219,POLÍGONO CENTRAL,20.698197,-103.342554,IN_SERVICE


In [8]:
info = ['Número óptimo bicicletas: '+str(x) for x in mejor_asignacion]

In [9]:
data2['asignacion'] = info

C:\Users\Alejandra Velasco\AppData\Local\Temp\ipykernel_15740\2605004432.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['asignacion'] = info


In [10]:
data2

,index,id,name,obcn,location,latitude,longitude,status,asignacion
0,0,2,(GDL-001) C. Epigmenio Glez./ Av. 16 de Sept.,GDL-001,POLÍGONO CENTRAL,20.666378,-103.348820,IN_SERVICE,Número óptimo bicicletas: 5
1,1,3,(GDL-002) C. Colonias / Av. Niños héroes,GDL-002,POLÍGONO CENTRAL,20.667228,-103.366000,IN_SERVICE,Número óptimo bicicletas: 5
2,2,4,(GDL-003) C. Vidrio / Av. Chapultepec,GDL-003,POLÍGONO CENTRAL,20.667690,-103.368252,IN_SERVICE,Número óptimo bicicletas: 4
3,3,5,(GDL-004) C. Ghilardi /C. Miraflores,GDL-004,POLÍGONO CENTRAL,20.691750,-103.362550,IN_SERVICE,Número óptimo bicicletas: 5
4,4,6,(GDL-005) C. San Diego /Calzada Independencia,GDL-005,POLÍGONO CENTRAL,20.681158,-103.339363,IN_SERVICE,Número óptimo bicicletas: 3
...,...,...,...,...,...,...,...,...,...
295,307,323,(GDL-213) C. Palavicini / Av. Circunvalación,GDL-213,POLÍGONO CENTRAL,20.704670,-103.345589,IN_SERVICE,Número óptimo bicicletas: 6
296,308,324,(GDL-221) De los científicos/ J. J. Tablada,GDL-221,POLÍGONO CENTRAL,20.700192,-103.345961,IN_SERVICE,Número óptimo bicicletas: 4
297,309,325,(GDL-226) C. Luis G. Cuevas /Av. Revolución,GDL-226,TLQ-CORREDORATLAS,20.652408,-103.312902,IN_SERVICE,Número óptimo bicicletas: 5
298,310,326,(GDL-219) C. D. Rivera / C. Filósofos,GDL-219,POLÍGONO CENTRAL,20.698197,-103.342554,IN_SERVICE,Número óptimo bicicletas: 4


In [13]:
!pip install folium
import folium

     ---------------------------------------- 0.0/102.3 kB ? eta -:--:--
     ----------------------- --------------- 61.4/102.3 kB 1.6 MB/s eta 0:00:01
     -------------------------------------- 102.3/102.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
lat = 20.66682
lon = -103.39182

m = folium.Map(location=(lat,lon),zoom_start=12)
icon_path = 'data/bike1.ico'

for i in range(len(data2['latitude'])):
    lat = data2['latitude'][i]
    lon = data2['longitude'][i]
    info = data2['asignacion'].loc[i]
    custom_icon = folium.CustomIcon(
        icon_image=icon_path,
        icon_size=(30, 30)  # Tamaño personalizado del icono (ajusta según tus necesidades)
    )

    mark = folium.Marker(location=(lat, lon), popup=info, icon=custom_icon)
    mark.add_to(m)

In [ ]:
m